# Давайте строить модель

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns

%matplotlib inline

In [2]:
raw_data = pd.read_csv("data/Moscow_weather_preparsed.csv").drop(columns=["Unnamed: 0"])
raw_data.head()

,DATE,TEMP,TEMP_Count,DEWP,DEWP_Count,SLP,SLP_Count,STP,STP_Count,VISIB,...,MAX_is_missing,MIN_is_missing,PRCP_is_missing,SNDP_is_missing,Is_Fog,Is_Rain or Drizzle,Is_Snow or Ice Pellets,Is_Hail,Is_Thunder,Is_Tornado or Funnel Cloud
0,1936-12-31,16.0,4.0,9999.9,0.0,1022.9,4.0,9999.9,0.0,3.0,...,0,0,0,1,0,0,0,0,0,0
1,1937-01-01,33.5,4.0,9999.9,0.0,1020.9,4.0,9999.9,0.0,2.5,...,0,0,0,1,0,1,0,0,0,0
2,1937-01-02,28.8,4.0,9999.9,0.0,1020.4,4.0,9999.9,0.0,4.0,...,0,0,0,1,0,0,0,0,0,0
3,1937-01-03,29.0,4.0,9999.9,0.0,1013.2,4.0,9999.9,0.0,2.5,...,0,0,0,1,0,1,1,0,0,0
4,1937-01-04,30.0,4.0,9999.9,0.0,1013.2,4.0,9999.9,0.0,2.2,...,0,0,0,1,0,0,1,0,0,0


In [3]:
data = raw_data.drop(columns=['STP', 'STP_Count', 'MAX_Flag', 'MIN_Flag', 'PRCP_Flag'])
data.head()

,DATE,TEMP,TEMP_Count,DEWP,DEWP_Count,SLP,SLP_Count,VISIB,VISIB_Count,WDSP,...,MAX_is_missing,MIN_is_missing,PRCP_is_missing,SNDP_is_missing,Is_Fog,Is_Rain or Drizzle,Is_Snow or Ice Pellets,Is_Hail,Is_Thunder,Is_Tornado or Funnel Cloud
0,1936-12-31,16.0,4.0,9999.9,0.0,1022.9,4.0,3.0,4.0,9.0,...,0,0,0,1,0,0,0,0,0,0
1,1937-01-01,33.5,4.0,9999.9,0.0,1020.9,4.0,2.5,4.0,13.0,...,0,0,0,1,0,1,0,0,0,0
2,1937-01-02,28.8,4.0,9999.9,0.0,1020.4,4.0,4.0,4.0,999.9,...,0,0,0,1,0,0,0,0,0,0
3,1937-01-03,29.0,4.0,9999.9,0.0,1013.2,4.0,2.5,4.0,17.0,...,0,0,0,1,0,1,1,0,0,0
4,1937-01-04,30.0,4.0,9999.9,0.0,1013.2,4.0,2.2,4.0,13.3,...,0,0,0,1,0,0,1,0,0,0


In [4]:
goal_features = list(data.columns[-6:])
# попробуем оставить только адекватные фичи
# все Count скорее всего можно игнорировать
# пропущенные значения и так достаточно отделены от возможных данных
# work_features = ["TEMP", "DEWP", "SLP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN", "PRCP", "SNDP"]
work_features = ["TEMP", "DEWP", "SLP", "VISIB", "WDSP", "MXSPD", "GUST", "MAX", "MIN", "PRCP", "SNDP"]
# work_features += [s + "_is_missing" for s in work_features]

In [5]:
# давайте учитывать данные до 3 дней в прошлое
max_days_back = 3
data_prev = {}
goal = data[goal_features]
cur_data = data[work_features]
cur_res = pd.DataFrame(index=data.index)
data_size = data.shape[0]
for i in range(1, max_days_back + 1):
    # индексы - боль
    cur_data = cur_data.drop(index=cur_data.index[-1]).set_index(np.arange(data_size - i))
    goal = goal.drop(index=goal.index[0]).set_index(np.arange(data_size - i))
    cur_res = cur_res.drop(index=cur_res.index[0]).set_index(np.arange(data_size - i))
    for feature in work_features:
        cur_res["{} {} day(s) ago".format(feature, i)] = cur_data[feature]
    data_prev[i] = (cur_res, goal)

In [6]:
data_prev[3][0][["TEMP 1 day(s) ago", "TEMP 2 day(s) ago", "TEMP 3 day(s) ago"]].head()

,TEMP 1 day(s) ago,TEMP 2 day(s) ago,TEMP 3 day(s) ago
0,28.8,33.5,16.0
1,29.0,28.8,33.5
2,30.0,29.0,28.8
3,28.0,30.0,29.0
4,26.5,28.0,30.0


In [7]:
print(*list(data_prev[1][1]["Is_Rain or Drizzle"][:50]))
print(*list(data_prev[2][1]["Is_Rain or Drizzle"][:50]))
print(*list(data_prev[3][1]["Is_Rain or Drizzle"][:50]))
print()
print(*list(data_prev[1][1]["Is_Rain or Drizzle"][-50:]))
print(*list(data_prev[2][1]["Is_Rain or Drizzle"][-50:]))
print(*list(data_prev[3][1]["Is_Rain or Drizzle"][-50:]))

# выглядит адекватно

1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0

0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0
0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0
0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0


### Теперь можно попробовать разные модели:

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [9]:
X_train, X_test, y_train, y_test = train_test_split(*data_prev[1])

In [10]:
y_train["Is_Tornado or Funnel Cloud"].value_counts()

0    17106
Name: Is_Tornado or Funnel Cloud, dtype: int64

In [11]:
# не было ещё у нас в Москве торнадо, можно выкинуть
goal_features = goal_features[:5]

In [12]:
# Поскольку разбиваем на 2 класса, score - просто доля правильных ответов

def test_clfs(X, y, goal_features, clfs, names, verbose=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    bests = []
    for goal in goal_features:
        if verbose:
            print("Predicting {}:".format(goal))
        goal_train, goal_test = y_train[goal], y_test[goal]
        best = (-1, None)
        for clf, name in zip(clfs, names):
            if verbose:
                print("{}:".format(name))
            clf.fit(X_train, goal_train)
            score = clf.score(X_test, goal_test)
            max_class = np.max(goal_train.value_counts() / goal_train.shape[0])
            if score >= best[0]:
                best = (score, goal, name, clf)
            if verbose:
                print("max class: {}".format(max_class))
                print("score: {} ({})".format(score, "OK" if score > max_class else "BAD"))
                print("-------------------------------------------------------------")
        bests.append(best)
        if verbose:
            print("#############################################################")
    return bests

In [13]:
clfs = [LogisticRegression(random_state=123), LinearSVC(random_state=123),
        BernoulliNB(), DecisionTreeClassifier(random_state=123)]
names = ["Log reg", "Lin SVM", "Bayes (bernoulli)", "Decision tree"]

bests = test_clfs(*data_prev[1], goal_features, clfs, names)

In [14]:
bests
#Пока что наивный байес работает лучше всех

[(0.97790636507101525,
  'Is_Fog',
  'Log reg',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=123, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False)),
 (0.68683149219708928,
  'Is_Rain or Drizzle',
  'Bayes (bernoulli)',
  BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)),
 (0.81483429773803262,
  'Is_Snow or Ice Pellets',
  'Decision tree',
  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=123,
              splitter='best')),
 (0.99491495704015431,
  'Is_Hail',
  'Bayes (bernoulli)',
  BernoulliNB(alpha=1.0, binarize=0.0, cl

In [15]:
# добавим ансамблей
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

clfs = [LogisticRegression(random_state=123), BernoulliNB(), RandomForestClassifier(random_state=123)]
names = ["Log reg", "Bayes (bernoulli)", "Random forest"]

bests = test_clfs(*data_prev[3], goal_features, clfs, names)

In [16]:
bests
# случайный лес ещё лучше

[(0.97334268677656965,
  'Is_Fog',
  'Random forest',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=123, verbose=0, warm_start=False)),
 (0.69344089793055064,
  'Is_Rain or Drizzle',
  'Random forest',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=123, verbose=0, warm_start=False)),
 (0.84128376008418104,
  'Is_Snow or

In [17]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

clfs = [RandomForestClassifier(random_state=123),
        AdaBoostClassifier(LogisticRegression(random_state=123), n_estimators=5, random_state=123),
        BernoulliNB(), AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), random_state=123)]
names = ["Random forest", "Log reg (boosted)", "Bayes (bernoulli, boosted)", "Decision tree (boosted)"]

bests = test_clfs(*data_prev[3], goal_features, clfs, names)
# байес умирает после бустинга

In [18]:
bests

[(0.97790249035426169,
  'Is_Fog',
  'Random forest',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=123, verbose=0, warm_start=False)),
 (0.70641880042090499,
  'Is_Rain or Drizzle',
  'Random forest',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=123, verbose=0, warm_start=False)),
 (0.83602244826376715,
  'Is_Snow or

### Теперь оптимизируем параметры оптимальных моделей

In [19]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

params = {'n_estimators': [40, 50, 60, 70, 80], 
          'max_depth': list(range(2, 9))}

#optimiser = RandomizedSearchCV(RandomForestClassifier(), params, refit=False, n_iter=10)
optimiser = GridSearchCV(RandomForestClassifier(random_state=123), params, refit=False)
X, y = data_prev[1]
%time optimiser.fit(X, y["Is_Snow or Ice Pellets"])
print(optimiser.best_params_)
print(optimiser.best_score_)

Wall time: 1min 10s
{'max_depth': 7, 'n_estimators': 80}
0.854268052085


In [20]:
# при кросс-валидации случайный лес оказывается всё-таки лучше всех
clfs = [RandomForestClassifier(random_state=123),
        RandomForestClassifier(random_state=123),
        RandomForestClassifier(random_state=123),
        RandomForestClassifier(random_state=123),
        RandomForestClassifier(random_state=123)]
clf_params = [{'n_estimators': list(range(50, 100, 10)), 
               'max_depth': list(range(2, 5)) + [None]},

              {'n_estimators': list(range(30, 80, 10)), 
               'max_depth': list(range(5, 10)) + [None]},

              {'n_estimators': list(range(30, 80, 10)), 
               'max_depth': list(range(5, 10)) + [None]},

              {'n_estimators': list(range(5, 30, 5)), 
               'max_depth': list(range(3, 7)) + [None]},

              {'n_estimators': list(range(5, 30, 5)), 
               'max_depth': list(range(5, 10)) + [None]}]

X, y = data_prev[3]
bests = []
for goal, clf, params in zip(goal_features, clfs, clf_params):
    print("Optimising {}:".format(goal))
    y_goal = y[goal]
    optimiser = GridSearchCV(clf, params, refit=False)
    %time optimiser.fit(X, y_goal)
    print("Params:", optimiser.best_params_)
    print("Best score:", optimiser.best_score_)
    bests.append((optimiser.best_params_, optimiser.best_score_))
    print("#############################################################")

Optimising Is_Fog:
Wall time: 1min 8s
Params: {'max_depth': 2, 'n_estimators': 90}
Best score: 0.900381461832
#############################################################
Optimising Is_Rain or Drizzle:
Wall time: 1min 51s
Params: {'max_depth': 8, 'n_estimators': 50}
Best score: 0.708905160696
#############################################################
Optimising Is_Snow or Ice Pellets:
Wall time: 1min 42s
Params: {'max_depth': 8, 'n_estimators': 70}
Best score: 0.851756039812
#############################################################
Optimising Is_Hail:
Wall time: 18.3 s
Params: {'max_depth': 3, 'n_estimators': 5}
Best score: 0.993817687552
#############################################################
Optimising Is_Thunder:
Wall time: 29.3 s
Params: {'max_depth': 8, 'n_estimators': 20}
Best score: 0.959836892182
#############################################################


In [21]:
bests

[({'max_depth': 2, 'n_estimators': 90}, 0.90038146183189371),
 ({'max_depth': 8, 'n_estimators': 50}, 0.7089051606962774),
 ({'max_depth': 8, 'n_estimators': 70}, 0.85175603981233827),
 ({'max_depth': 3, 'n_estimators': 5}, 0.99381768755206734),
 ({'max_depth': 8, 'n_estimators': 20}, 0.95983689218222479)]

In [30]:
goal_data = data_prev[3][1]
randoms = [np.max(np.max(goal_data[goal].value_counts() / goal_data[goal].shape[0])) for goal in goal_features]
randoms

[0.9743499802692156,
 0.68290437146490113,
 0.71587670452054197,
 0.99381768755206734,
 0.95974919980707674]

Немного лучше тупого нерандома; score для тумана возможно пострадал из-за кросс-валидации, т.к. для базового случайного леса было при одинарном разбиении выборки было 0.97790249035426169, а если train в разбиении почти не содержит единиц, может быть достаточно плохо